<a href="https://colab.research.google.com/github/RaghavGanesh7/Coding-AIAgents/blob/main/Langgraph_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the langgraph library
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 19.2 MB/s eta 0:00:00


# New Section

In [2]:
# Import necessary modules from typing and langgraph
from typing import TypedDict,List
from langgraph.graph import StateGraph, START, END

In [3]:
# Define a TypedDict for the agent's state
# This defines the structure of the state object that will be passed between nodes in the graph
class AgentState(TypedDict):
    name:str
    values: List[int]
    result : str

In [4]:
# Define a function to process the values in the agent's state
# This function takes the current state as input and updates it
def process_values(state:AgentState):
    # You can uncomment the print statement to see the state at this point
    #print(state)
    """function to process multiple inputs"""
    # Update the 'result' key in the state with a greeting and the sum of the values
    state['result'] = "Hi " + state['name'] + ". Your sum is " + str(sum(state['values']))
    return state

In [5]:
# Create a new StateGraph with the defined AgentState
graph = StateGraph(AgentState)
# Add a node named "processor" to the graph, using the process_values function
graph.add_node("processor", process_values)
# Set the entry point of the graph to the "processor" node
graph.set_entry_point("processor")
# Set the finish point of the graph to the "processor" node
graph.set_finish_point("processor")

# Compile the graph into a runnable application
app = graph.compile()

In [6]:
# Invoke the compiled graph with an initial state and print the 'result'
# This demonstrates running the simple graph with a starting state
app.invoke({"name":"Raghav","values":[1,2,3,4]})['result']

'Hi Raghav. Your sum is 10'

In [7]:
# Define a TypedDict for a router's state
# This state will hold information for routing based on an operator
class RouterState(TypedDict):
  operator : str
  num1 : int
  num2 : int
  result : str

In [8]:
# Define functions for different operations
# These functions take the RouterState and update the result based on the operation

def add_operation(state: RouterState):
  # Print the state to see the input
  print(state)
  # Perform addition and update the result
  state['result'] = str(state['num1'] + state['num2'])
  return state

def subtract_operation(state:RouterState):
  # Perform subtraction and update the result
  state['result'] = str(state['num1'] - state['num2'])
  return state

# Define a router function that determines the next node based on the 'operator'
def myrouter(state:RouterState):
  # Print the state to see the input
  print(state)
  # Return the name of the next node based on the operator
  if state['operator'] == '+':
    return "add_operation"
  elif state['operator'] == '-':
    return "subtract_operation"

In [9]:
# Create a new StateGraph for the router
graph = StateGraph(RouterState)
# Add nodes for the add and subtract operations
graph.add_node("add_node", add_operation)
graph.add_node("sub_node", subtract_operation)
# Add a router node that simply passes the state through (the routing logic is in add_conditional_edges)
graph.add_node("router", lambda state:state)

# Define the graph's edges
# Set the entry point to the router node
graph.add_edge(START, "router");
# Add conditional edges from the router based on the myrouter function
# The output of myrouter (e.g., "add_operation") determines the next node
graph.add_conditional_edges("router",myrouter,{
    "add_operation": "add_node",
    "subtract_operation": "sub_node"
})
# Add edges from the operation nodes to the end point
graph.add_edge("add_node", END)
graph.add_edge("sub_node",END)

# Compile the graph
app = graph.compile()

In [10]:
# Define an initial state for the router graph
initState = RouterState(operator='+',num1=1,num2=2);
# Invoke the compiled router graph with the initial state and print the result
app.invoke(initState)['result']

{'operator': '+', 'num1': 1, 'num2': 2}
{'operator': '+', 'num1': 1, 'num2': 2}


'3'

In [11]:
# Define a TypedDict for the state of a guessing game agent
class AgentState(TypedDict):
  player_name:str
  target: int
  guesses: List[int]
  attempts: int
  lower: int
  upper: int

In [12]:
# Define functions for the guessing game
# Setup function to initialize the game state
def setup(state:AgentState):
  state['attempts'] = 0
  state['lower'] = 0
  state['upper'] = 100
  state['guesses'] = []
  return state

# Guess function to make a guess based on the current range
def guess(state:AgentState):
  state['attempts'] += 1
  state['guesses'].append((state['lower'] + state['upper']) // 2)
  return state

# Hint node function to provide a hint and update the range
def hint_node(state:AgentState):
  # Print the state to see the current game progress
  print(state)
  # Update the lower or upper bound based on the guess
  if state['guesses'][-1] < state['target']:
    state['lower'] = state['guesses'][-1]
  elif state['guesses'][-1] > state['target']:
    state['upper'] = state['guesses'][-1]
  return state

# Function to determine if the game should continue
def should_continue(state:AgentState):
  # Check if the last guess is the target or if the maximum attempts are reached
  if state['guesses'][-1] == state['target']:
    return "continue"
  elif state['attempts'] > 10:
    return "continue"
  else:
    return "loop"

In [13]:
# Create a new StateGraph for the guessing game
graph = StateGraph(AgentState)
# Add nodes for setup, guessing (loop), hinting, and checking to continue
graph.add_node("setup",setup)
graph.add_node("loop",guess)
graph.add_node("hint",hint_node)
# The should_continue node here is just a placeholder as the logic is in add_conditional_edges
graph.add_node("should_continue",lambda x:x)

# Define the graph's edges
# Set the entry point to the setup node
graph.add_edge(START,"setup")
# Connect setup to the guessing loop
graph.add_edge("setup","loop")
# Connect the guessing loop to the hint node
graph.add_edge("loop","hint")
# Add conditional edges from the hint node based on the should_continue function
# The return value of should_continue determines if the game loops or ends
graph.add_conditional_edges("hint",should_continue,{
    "continue": END, # If should_continue returns "continue", the graph ends
    "loop": "loop" # If should_continue returns "loop", the graph goes back to the guess node
})
# Add an edge from hint to END (This edge seems redundant given the conditional edges, might be a leftover)
graph.add_edge("hint",END)

# Compile the graph
app = graph.compile()
# Define the initial state for the guessing game
init_state = AgentState(target=60,player_name="Raghav")
# Invoke the compiled graph with the initial state and print the final state
print(app.invoke(init_state))

{'player_name': 'Raghav', 'target': 60, 'guesses': [50], 'attempts': 1, 'lower': 0, 'upper': 100}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75], 'attempts': 2, 'lower': 50, 'upper': 100}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62], 'attempts': 3, 'lower': 50, 'upper': 75}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62, 56], 'attempts': 4, 'lower': 50, 'upper': 62}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62, 56, 59], 'attempts': 5, 'lower': 56, 'upper': 62}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62, 56, 59, 60], 'attempts': 6, 'lower': 59, 'upper': 62}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62, 56, 59, 60], 'attempts': 6, 'lower': 59, 'upper': 62}


In [14]:
# Install necessary libraries: langgraph and langchain_google_genai
# langgraph is for building agent graphs
# langchain_google_genai provides integration with Google's Generative AI models
!pip install langgraph

!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [15]:
# Import required modules
# ChatGoogleGenerativeAI for using Google's chat models
# TypedDict, List, Union for type hinting
# StateGraph, START, END for building the graph
# HumanMessage, AIMessage for representing messages in a conversation
# userdata for accessing Colab secrets (like API keys)
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict,List,Union
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from google.colab import userdata

In [16]:
# Define the AgentState for this graph, which contains a list of messages
class AgentState(TypedDict):
  messages: List[Union[HumanMessage,AIMessage]]

# Initialize the ChatGoogleGenerativeAI model
# Replace 'YOUR_API_KEY' with your actual API key or use userdata.get to access it from Colab secrets
llm  = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=userdata.get('GOOGLE_API_KEY'))

# Define a function to process the state using the LLM
def ex_process(state:AgentState):
  # Invoke the LLM with the current messages in the state
  response = llm.invoke(state['messages'])
  # Append the AI's response to the messages in the state
  state['messages'].append(AIMessage(content=response.content))
  # Print the AI's response
  print(response.content)
  return state

# Create a new StateGraph
graph = StateGraph(AgentState)
# Add a node named "ex_process" using the ex_process function
graph.add_node("ex_process", ex_process)
# Set the entry point to "ex_process"
graph.add_edge(START,"ex_process")
# Set the finish point to "ex_process"
graph.add_edge("ex_process",END)

# Compile the graph
agent = graph.compile()

# Initialize the conversation history
conversation_history = []

# Start an interactive loop for conversation
user_input = input("Enter :")

while(user_input != "exit"):
  # Append the user's input as a HumanMessage to the history
  conversation_history.append(HumanMessage(content=user_input))
  # Invoke the agent with the current conversation history
  state = agent.invoke({"messages":conversation_history})
  # Update the conversation history with the new state (including the AI's response)
  conversation_history = state['messages']
  # Get the next user input
  user_input = input("Enter :")
  # The commented line below is an alternative way to invoke the agent, but the loop above is used here
  #agent.invoke({"message":[HumanMessage(content="Hi")]})

KeyboardInterrupt: Interrupted by user

In [17]:
# Install the langchain library
!pip install langchain

In [18]:
# Import necessary modules for building an agent with tools
# ChatGoogleGenerativeAI for using Google's chat models
# Annotated, Sequence, TypedDict, List, Union for type hinting
# StateGraph, START, END for building the graph
# tool for defining tools
# add_messages for handling message history in the state
# ToolNode for creating a node that executes tools
# HumanMessage, AIMessage, BaseMessage, ToolMessage, SystemMessage for different message types
# userdata for accessing Colab secrets
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated,Sequence,TypedDict,List,Union
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.messages import BaseMessage
from langchain_core.messages import ToolMessage
from langchain_core.messages import SystemMessage
from google.colab import userdata

In [19]:
# Define the AgentState for this graph, which includes message history
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]

# Define tools that the agent can use
@tool
def add(a:int, b:int):
  """This is a addition function to add two numbers"""
  return a+b

def multiply(a:int, b:int):
  """This is a multiplication function to multiply two numbers"""
  return a*b

# List of available tools
tools = [add,multiply]
# Initialize the LLM and bind the tools to it
llm  = ChatGoogleGenerativeAI(model="gemini-2.5-pro", api_key=userdata.get('GOOGLE_API_KEY')).bind_tools(tools)

# Define a node that calls the LLM
def mode_call(state:AgentState):
  # Define a system prompt for the AI's role
  system_prompt = SystemMessage(content="You a AI Agent. Answer to best of your ability")
  # Invoke the LLM with the system prompt and the current messages
  response = llm.invoke([system_prompt] + state["messages"])
  # Print the AI's response
  print(response.content)
  # Return the state with the AI's response added
  return {'messages': [response]}

# Define a function to determine if the graph should continue based on tool calls
def should_continue(state:AgentState):
  messages = state['messages']
  last_message = messages[-1]
  # If the last message has no tool calls, end the graph
  if not last_message.tool_calls:
    return "end"
  # If the last message has tool calls, continue to the tools node
  else:
    return "continue"

# Create a new StateGraph
graph = StateGraph(AgentState)
# Add the LLM call node
graph.add_node("mode_call", mode_call)
# Add the tool node, which will execute the tool calls
tool_node = ToolNode(tools=tools)
graph.add_node("tools",tool_node)
# Add conditional edges from the mode_call node based on the should_continue function
# If should_continue returns "continue", go to the "tools" node
# If should_continue returns "end", go to the END point
graph.add_conditional_edges("mode_call",should_continue,{
    "continue": "tools",
    "end": END
})
# Set the entry point to the mode_call node
graph.set_entry_point("mode_call")
# Add an edge from the tools node back to the mode_call node to get the LLM's response to the tool results
graph.add_edge("tools","mode_call")

# Compile the graph
app = graph.compile()

# Define a helper function to print the streamed output
def print_stream(stream):
  for s in stream:
    message = s['messages'][-1]
    if isinstance(message, tuple):
      print(message)
    else:
      message.pretty_print()

# Define the initial input for the agent
inputs = {"messages":[("user","Add 5+5. And then multiply the result with 5. Also who created you?")]}

# Run the agent with streaming and print the output
print_stream(app.stream(inputs,stream_mode="values"))

================================ Human Message =================================

Add 5+5. And then multiply the result with 5. Also who created you?

================================== Ai Message ==================================
Tool Calls:
  add (9fc37050-c5cc-4b06-b086-f8865ad96ad2)
 Call ID: 9fc37050-c5cc-4b06-b086-f8865ad96ad2
  Args:
    a: 5
    b: 5
================================= Tool Message =================================
Name: add

10

================================== Ai Message ==================================
Tool Calls:
  multiply (d1b7b1bf-4da2-4dcf-9996-41fb5d54388b)
 Call ID: d1b7b1bf-4da2-4dcf-9996-41fb5d54388b
  Args:
    a: 10
    b: 5
================================= Tool Message =================================
Name: multiply

50
I am a large language model, trained by Google.
================================== Ai Message ==================================

I am a large language model, trained by Google.
Tool Calls:
  add (7e1168fe-60b9-49db-bda5-3c2c

In [22]:
# Initialize a global variable to store the document content
document_content = ""

# Define the AgentState for this document agent, including message history
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Define tools for the document agent

@tool
def update(content: str) -> str:
    """Updates the document with the provided content."""
    global document_content
    document_content = content
    return f"Document has been updated successfully! The current content is:\n{document_content}"

@tool
def save(filename: str) -> str:
    """Save the current document to a text file and finish the process.

    Args:
        filename: Name for the text file.
    """

    global document_content

    # Ensure the filename ends with .txt
    if not filename.endswith('.txt'):
        filename = f"{filename}.txt"

    try:
        # Write the document content to the specified file
        with open(filename, 'w') as file:
            file.write(document_content)
        print(f"\n💾 Document has been saved to: {filename}")
        return f"Document has been saved successfully to '{filename}'."

    except Exception as e:
        return f"Error saving document: {str(e)}"

# List of available tools for the agent
tools = [update, save]

# Initialize the ChatGoogleGenerativeAI model and bind the tools
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro", api_key=userdata.get('GOOGLE_API_KEY')).bind_tools(tools)

# Define the main agent function
def our_agent(state: AgentState) -> AgentState:
    # Define the system prompt for the agent
    system_prompt = SystemMessage(content=f"""
    You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.

    - If the user wants to update or modify content, use the 'update' tool with the complete updated content.
    - If the user wants to save and finish, you need to use the 'save' tool.
    - Make sure to always show the current document state after modifications.

    The current document content is:{document_content}
    """)

    # Handle the initial state where there are no messages
    if not state["messages"]:
        user_input = "I'm ready to help you update a document. What would you like to create?"
        user_message = HumanMessage(content=user_input)
    else:
        # Get user input during the conversation
        user_input = input("\nWhat would you like to do with the document? ")
        print(f"\n👤 USER: {user_input}")
        user_message = HumanMessage(content=user_input)

    # Combine system prompt, existing messages, and new user message
    all_messages = [system_prompt] + list(state["messages"]) + [user_message]

    # Invoke the model with all messages
    response = model.invoke(all_messages)

    # Print the AI's response and indicate if tools are being used
    print(f"\n🤖 AI: {response.content}")
    if hasattr(response, "tool_calls") and response.tool_calls:
        print(f"🔧 USING TOOLS: {[tc['name'] for tc in response.tool_calls]}")

    # Return the updated state with the user message and AI response
    return {"messages": list(state["messages"]) + [user_message, response]}

# Define a function to determine if the conversation should continue
def should_continue(state: AgentState) -> str:
    """Determine if we should continue or end the conversation."""

    messages = state["messages"]

    # Continue if there are no messages (initial state)
    if not messages:
        return "continue"

    # This looks for the most recent tool message....
    for message in reversed(messages):
        # ... and checks if this is a ToolMessage resulting from a successful save
        if (isinstance(message, ToolMessage) and
            "saved" in message.content.lower() and
            "document" in message.content.lower()):
            # If a save tool message is found, end the graph
            return "end" # goes to the end edge which leads to the endpoint

    # Otherwise, continue the conversation
    return "continue"

# Define a helper function to print messages in a readable format
def print_messages(messages):
    """Function I made to print the messages in a more readable format"""
    if not messages:
        return

    # Print the last 3 messages
    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"\n🛠️ TOOL RESULT: {message.content}")


# Create a new StateGraph
graph = StateGraph(AgentState)

# Add the agent and tools nodes
graph.add_node("agent", our_agent)
graph.add_node("tools", ToolNode(tools))

# Set the entry point to the agent node
graph.set_entry_point("agent")

# Add an edge from the agent to the tools node
graph.add_edge("agent", "tools")

# Add conditional edges from the tools node based on the should_continue function
# If should_continue returns "continue", go back to the "agent" node
# If should_continue returns "end", go to the END point
graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        "continue": "agent",
        "end": END,
    },
)

# Compile the graph
app = graph.compile()

# Define the main function to run the document agent
def run_document_agent():
    print("\n ===== DRAFTER =====")

    # Initialize the state
    state = {"messages": []}

    # Stream the output of the graph and print messages
    for step in app.stream(state, stream_mode="values"):
        if "messages" in step:
            print_messages(step["messages"])

    print("\n ===== DRAFTER FINISHED =====")

# Run the document agent when the script is executed
if __name__ == "__main__":
    run_document_agent()


 ===== DRAFTER =====

🤖 AI: I will start with a simple document.

This is a document about my cat.

First, please add a title to this document.

What would you like to do with the document? My cats name is bangu

👤 USER: My cats name is bangu

🤖 AI: 
🔧 USING TOOLS: ['update']

🛠️ TOOL RESULT: Document has been updated successfully! The current content is:
My Cat Bangu

This is a document about my cat.

What would you like to do with the document? save the document

👤 USER: save the document

🤖 AI: What file name should I use?

🛠️ TOOL RESULT: Document has been updated successfully! The current content is:
My Cat Bangu

This is a document about my cat.

🛠️ TOOL RESULT: Document has been updated successfully! The current content is:
My Cat Bangu

This is a document about my cat.

What would you like to do with the document? cat.txt

👤 USER: cat.txt

🤖 AI: 
🔧 USING TOOLS: ['save']

💾 Document has been saved to: cat.txt

🛠️ TOOL RESULT: Document has been saved successfully to 'cat.txt'.



In [11]:
# Install necessary libraries for RAG (Retrieval Augmented Generation)
# langchain_community for various components like document loaders
# pypdf for loading PDF documents
# langchain_chroma for using Chroma as a vector store
!pip install langchain_community pypdf langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 81.4 MB/s eta 0:0

In [ ]:
# Import necessary modules
# ChatGoogleGenerativeAI for the LLM
# TypedDict, List, Union for type hinting
# StateGraph, START, END for building the graph
# tool for defining tools
# add_messages for handling message history
# ToolNode for executing tools
# HumanMessage, AIMessage, BaseMessage, ToolMessage, SystemMessage for message types
# userdata for accessing Colab secrets
# GoogleGenerativeAIEmbeddings for creating embeddings
# PyPDFLoader for loading PDF documents
# RecursiveCharacterTextSplitter for splitting text
# Chroma for the vector store
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated,Sequence,TypedDict,List,Union
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, ToolMessage, SystemMessage
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma


# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro",
 api_key=userdata.get('GOOGLE_API_KEY'),temperature=0)

# Initialize the embeddings model
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=userdata.get('GOOGLE_API_KEY'))

# Specify the document name
document_name = "Stock_Market_Performance_2024.pdf"

# Check if the document exists
if not os.path.exists(document_name):
  raise FileNotFoundError(f"PDF file not found")

# Initialize the PDF loader
pdf_loader = PyPDFLoader(document_name)

# Load the pages from the PDF document
try:
  pages = pdf_loader.load()
except Exception as e:
  raise Exception(f"Error loading PDF: {e}")

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the loaded pages into chunks
pages_split = text_splitter.split_documents(pages)

# Specify the directory and collection name for the Chroma vector store
persist_directory = "/content/"
collection_name = "stockmarket"

# Create the persist directory if it doesn't exist
if not os.path.exists(persist_directory):
  os.mkdir(persist_directory)

# Create or load the Chroma vector store from the document chunks
try:
  vector_store = Chroma.from_documents(
    documents=pages_split,
    embedding=embeddings,
    persist_directory=persist_directory,
    collection_name=collection_name
  )
  print(f"Created ChromaDB vector store")
except Exception as e:
  raise Exception

# Create a retriever from the vector store
retriever = vector_store.as_retriever(search_type="similarity",search_kwargs={"k":5})

# Define a tool for retrieving information from the document
@tool
def retriever_tool(query:str)->str:
  """
  This tool searches and returns the information from the Stock Information document.
  """
  # Invoke the retriever with the user's query
  docs = retriever.invoke(query)

  # Check if any relevant documents were found
  if not docs:
    return "I found no relevant information in the Stock Market Performance 2024 document."
  # Format the retrieved document content
  results = []
  for i,doc in enumerate(docs):
    results.append(f"Document {i+1}:\n{doc.page_content}")
  return "\n".join(results)

# List of available tools for the agent
tools = [retriever_tool]

# Bind the tools to the LLM
llm = llm.bind_tools(tools)

# Define the AgentState for this RAG agent
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]

# Define a function to determine if the graph should continue based on tool calls
def should_continue(state:AgentState):
  """
  Checks if the last message contains tool calls
  """
  result = state["messages"][-1]
  # Return "continue" if there are tool calls, otherwise return "end"
  if hasattr(result,'tool_calls') and len(result.tool_calls)>0:
    return "continue"
  else:
    return "end"

# Define the system prompt for the RAG agent
system_prompt = """
You are an intelligent AI assistant who answers questions about Stock Market Performance in 2024 based on the PDF document loaded into your knowledge base.
Use the retriever tool available to answer questions about the stock market performance data. You can make multiple calls if needed.
If you need to look up some information before asking a follow up question, you are allowed to do that!
Please always cite the specific parts of the documents you use in your answers.
"""

# Create a dictionary of tools for easy access by name
tools_dict = {our_tool.name: our_tool for our_tool in tools}

# Define a node that calls the LLM
def call_llm(state:AgentState):
  # Get the current messages and add the system prompt
  messages = list(state["messages"])
  messages = [SystemMessage(content=system_prompt)] + messages
  # Invoke the LLM
  messages = llm.invoke(messages)
  # Return the state with the LLM's response
  return {'messages':messages}

# Define a node that executes tool calls
def take_action(state:AgentState) -> AgentState:
  """Execute tool calls from LLM's response"""
  # Get the tool calls from the last message
  tool_calls = state["messages"][-1].tool_calls
  tool_messages = []
  # Iterate through each tool call
  for t in tool_calls:
    print(f"Calling tools:{t['name']} with query {t['args'].get('query')}")
    # Check if the tool exists
    if not t['name'] in tools_dict:
      print(f"Tool {t['name']} not found")
      tool_messages.append(ToolMessage(tool_call_id=t['id'],content="Incorrect tool name received",name=t['name']))
    else:
      # Invoke the tool and get the result
      result = tools_dict[t['name']].invoke(t['args'].get('query',''))
      print("Result length",str(len(result)))
      # Append the tool result as a ToolMessage
      tool_messages.append(ToolMessage(tool_call_id=t['id'],content=str(result),name=t['name']))

  print("Tool execution completed")
  # Return the state with the tool messages
  return {"messages":tool_messages }

# Create a new StateGraph
graph = StateGraph(AgentState)
# Add the LLM call node and the retriever agent (tool execution) node
graph.add_node("call_llm",call_llm)
graph.add_node("retriever_agent",take_action)
# Add conditional edges from the call_llm node based on should_continue
# If should_continue returns "continue", go to the retriever_agent
# If should_continue returns "end", go to the END point
graph.add_conditional_edges("call_llm",
should_continue
,{
    "continue": "retriever_agent",
    "end": END
})

# Add an edge from the retriever_agent back to call_llm to process the tool results
graph.add_edge("retriever_agent","call_llm")
# Set the entry point to the call_llm node
graph.set_entry_point("call_llm")

# Compile the graph
rag_agent = graph.compile()

# Define the main function to run the RAG agent
def running_agent():
    print("\n=== RAG AGENT===")

    # Start an interactive loop for questions
    while True:
        user_input = input("\nWhat is your question: ")
        # Exit the loop if the user types 'exit' or 'quit'
        if user_input.lower() in ['exit', 'quit']:
            break

        # Create a HumanMessage from the user input
        messages = [HumanMessage(content=user_input)] # converts back to a HumanMessage type

        # Invoke the RAG agent with the user's message
        result = rag_agent.invoke({"messages": messages})

        # Print the final answer from the agent
        print("\n=== ANSWER ===")
        print(result['messages'][-1].content)

# Run the RAG agent when the script is executed
running_agent()

Exception: Error loading PDF: `pypdf` package not found, please install it with `pip install pypdf`

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated,Sequence,TypedDict,List,Union
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, ToolMessage, SystemMessage
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import os


In [10]:
from genericpath import exists
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro",
 api_key=userdata.get('GOOGLE_API_KEY'),temperature=0)

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=userdata.get('GOOGLE_API_KEY'))

document_name = "Stock_Market_Performance_2024.pdf"

if not os.path.exists(document_name):
  raise FileNotFoundError(f"PDF file not found")


pdf_loader = PyPDFLoader(document_name)

try:
  pages = pdf_loader.load()
except Exception as e:
  raise Exception(f"Error loading PDF: {e}")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

pages_split = text_splitter.split_documents(pages)


persist_directory = "/content/"
collection_name = "stockmarket"

if not os.path.exists(persist_directory):
  os.mkdir(persist_directory)

try:
  vector_store = Chroma.from_documents(
    documents=pages_split,
    embedding=embeddings,
    persist_directory=persist_directory,
    collection_name=collection_name
  )
  print(f"Created ChromaDB vector store")
except Exception as e:
  raise Exception

retriever = vector_store.as_retriever(search_type="similarity",search_kwargs={"k":5})

@tool
def retriever_tool(query:str)->str:
  """
  This tool searches and returns the information from the Stock Information document.
  """
  docs = retriever.invoke(query)

  if not docs:
    return "I found no relevant information in the Stock Market Performance 2024 document."
  results = []
  for i,doc in enumerate(docs):
    results.append(f"Document {i+1}:\n{doc.page_content}")
  return "\n".join(results)

tools = [retriever_tool]

llm = llm.bind_tools(tools)

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]

def should_continue(state:AgentState):
  """
  Checks if the last message contains tool calls
  """
  result = state["messages"][-1]
  return hasattr(result,'tool_calls') and len(result.tool_calls)>0

system_prompt = """
You are an intelligent AI assistant who answers questions about Stock Market Performance in 2024 based on the PDF document loaded into your knowledge base.
Use the retriever tool available to answer questions about the stock market performance data. You can make multiple calls if needed.
If you need to look up some information before asking a follow up question, you are allowed to do that!
Please always cite the specific parts of the documents you use in your answers.
"""

tools_dict = {our_tool.name: our_tool for our_tool in tools}

def call_llm(state:AgentState):
  messages = list(state["messages"])
  messages = [SystemMessage(content=system_prompt)] + messages
  messages = llm.invoke(messages)
  return {'messages':messages}


def take_action(state:AgentState) -> AgentState:
  """Execute tool calls from LLM's response"""
  tool_calls = state["messages"][-1].tool_calls
  results = []
  for t in tool_calls:
    print(f"Calling tools:{t['name']} with query {t['args'].get('query')}")
    if not t['name'] in tools_dict:
      print(f"Tool {t['name']} not found")
      result = "Incorrect tool name received"
    else:
      result = tools_dict[t['name']].invoke(t['args'].get('query',''))
      print("Result length",str(len(result)))

    results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))

  print("Tool execution completed")
  return {"messages":result }

graph = StateGraph(AgentState)
graph.add_node("call_llm",call_llm)
graph.add_node("retriever_agent",take_action)
graph.add_conditional_edges("call_llm",
should_continue,
{True: "retriever_agent", False: END})

graph.add_edge("retriever_agent","call_llm")
graph.set_entry_point("call_llm")

rag_agent = graph.compile()
def running_agent():
    print("\n=== RAG AGENT===")

    while True:
        user_input = input("\nWhat is your question: ")
        if user_input.lower() in ['exit', 'quit']:
            break

        messages = [HumanMessage(content=user_input)] # converts back to a HumanMessage type

        result = rag_agent.invoke({"messages": messages})

        print("\n=== ANSWER ===")
        print(result['messages'][-1].content)


running_agent()







Created ChromaDB vector store

=== RAG AGENT===

What is your question: which was best performing sector during 2024?
Calling tools:retriever_tool with query best performing sector in 2024
Result length 4814
Tool execution completed

=== ANSWER ===
Based on the document "Stock Market Performance in 2024," the best-performing sector in 2024 was the technology sector. This is supported by the fact that the "tech-heavy Nasdaq Composite outpaced the broader market, jumping nearly 29% for the year" and that a "key theme was the dominance of mega-cap technology stocks" (Document 1).

What is your question: exit
